## Implementation of a simple inverted index using unigrams, bigrams ,trigrams and a positional inverted index of unigrams.


In [1]:
import requests
from bs4 import BeautifulSoup
import time
import string
import os
import re
import collections
import pandas as pd
from queue import PriorityQueue
from nltk import word_tokenize, regexp_tokenize, ngrams, FreqDist
from nltk.corpus import stopwords
import numpy as np
from urllib.parse import urlsplit
import xlsxwriter




1.  a) Implement a simple inverted indexer that consumes the corpus from HW2: Task 1 as input and produces an inverted index as an output. Your index will contain the document IDs and term frequency within the document. Term frequencies  are stored in these inverted lists: TERM à (docID, tf), (docID, tf), ... A TERM is defined as a word n-gram, where n = 1, 2, and 3. Therefore, you will have three inverted indexes, one for each value of n. 
    b) Store the number of terms in each document in a separate data structure.
    c) You may employ any concrete data structures convenient for the programming language
    you are using, as long as you can write them to disk and read them back in when you want
    to run some queries. 
    d) Generate a positional inverted index for unigrams with gaps

In [306]:
"""
Read documents in the corpus.
For each document - (1) update term frequency (2) update inverted list for each term in the document
"""
fpath = '/users/ankithakumari/Documents/CS6200/HW3/BFS'
files = os.listdir(fpath)

# Create data structures for storing inverted lists
docNum = 1
unigram_index = {}
bigram_index = {}
trigram_index ={}
unigram_tf = {}
# Data structures to store term frequencies in each document
doc_tf_unigram = {}
doc_tf_bigram = {}
doc_tf_trigram = {}

    

In [291]:
"""
Utility function that builds the index for a document
Input: The index data structure, document number, frequency distibution of tokens
Action: Updates the index with document number and term frequency of each term within that document
The index is a dictionary where each inverted list is as follows:
{term1: [doc1 tf1 doc2 tf2 ...]}
the document ids and term frequencies are stored as a list. even indices represent document id and odd indices
represent term frequency in each of the document preceding it.
"""
def buildIndex(index_ds, doc, tf_dist):
    for term, freq in tf_dist.items():
        if term not in index_ds.keys():
            index_ds[term] = [doc, freq]
        else:
            index_ds[term].append(doc)
            index_ds[term].append(freq)
            
"""
Build index with position of terms
The inverted list has the following format: 
{term1: {doc_id1 : [pos_1, pos_2 ...],  doc_id2:[pos_1, pos_2..] .. }}
"""
            
def buildIndexWithPos(index_ds, doc, terms):
    for term in FreqDist(ngrams(terms, 1)):
        if term not in index_ds.keys():
            index_ds[term] = {doc:[]}                     #initialize dictionary
        else:
            index_ds[term][doc] = []
    i = 1
    for item in ngrams(terms,1):
        index_ds[item][doc].append(i) 
        i += 1       # appends the word position for each term as encountered while parsing
        
    

In [307]:
for file in files:
    # read tokens from file
    f = open('BFS/'+file, 'r', encoding="utf-8")
    tokens = f.read().splitlines()
    f.close()
    # remove any words that start with hyphen
    tokens =[token.lower() for token in tokens if (re.match(r'^[a-zA-Z0-9][ A-Za-z0-9-]*$', token))]
    # create n-grams
    unigram = ngrams(tokens, 1)
    bigram = ngrams(tokens, 2)
    trigram = ngrams(tokens, 3)
    
    # store term frequencies
    doc_tf_unigram[docNum] = sum(FreqDist(unigram).values())
    doc_tf_bigram[docNum] = sum(FreqDist(bigram).values())
    doc_tf_trigram[docNum] = sum(FreqDist(trigram).values())
    
    # create inverted index
    buildIndex(bigram_index, docNum, FreqDist(ngrams(tokens, 2)))
    buildIndex(trigram_index, docNum, FreqDist(ngrams(tokens, 3)))
    buildIndex(unigram_tf, docNum, FreqDist(ngrams(tokens, 1)))
    buildIndexWithPos(unigram_index, docNum, tokens)     # create positional index for unigrams
    # increment document Number
    docNum += 1
       


In [339]:
# Encode positions with differences between consecutive document ids
for term in unigram_index.keys():              # for each term in the index
    for doc in unigram_index[term].keys():     # for each document in the list
        for i in reversed(range(1, len(unigram_index[term][doc]))):
            unigram_index[term][doc][i] =unigram_index[term][doc][i] - unigram_index[term][doc][i-1] # save the difference
            
    
    

### For writing the inverted indexes to files
workbook = xlsxwriter.Workbook('idx3.xlsx')
worksheet = workbook.add_worksheet()
row = 0

for term in trigram_index.keys():
    col = 0
    row += 1
    worksheet.write(row, col, str(term).strip("()"))
    posting = trigram_index[term]
    i = 0
    while col < len(posting)/2:
        col += 1
        worksheet.write(row, col, str(posting[i]) + ":" + str(posting[i+1]))
        i += 2
        

workbook.close()



workbook = xlsxwriter.Workbook('unigram_idx.xlsx')
worksheet = workbook.add_worksheet()
row = 1
for term in unigram_index.keys():
    col = 0
    worksheet.write(row, col, str(term).strip("()"))
    postings = unigram_index[term]
    col += 1
    for doc, positions in postings.items():
        worksheet.write(row, col, doc)
        worksheet.write(row, col+1, str(positions))
        worksheet.write(row,col+2, len(positions))
        row += 1
workbook.close()   
        


workbook = xlsxwriter.Workbook('Doc_TF_Table.xlsx')
worksheet = workbook.add_worksheet()
row = 1
col = 0
for doc, tf in doc_tf_unigram.items():
    worksheet.write(row, col, doc)
    worksheet.write(row, col+1, tf)
    row += 1
workbook.close()

## TASK 2
a) Implement a program that uses a positional inverted index in Task 1-d) above to retrieve
the list of documents that contain a pair of terms within a proximity window k.
b) Using the positional index created in Task 1-d) above:
i. Find the list of documents that contain both space and mission within k = 6 and k
= 12 unigram tokens of each other. [Here, with k tokens implies there are at most
k tokens between the two terms, and the terms may appear in any order.]
ii. Find the list of documents that contain both earth and orbit within k = 5 and k = 10
unigram tokens of each other.
Note 1: Terms are not case-sensitive.
Note 2: You will produce 4 lists of Document IDs in total


In [348]:
# Decode the encoded positional indices
def getposition(pos_list):
    return [sum(pos_list[0:i]) for i in range(1, len(pos_list)+1)]

In [405]:
# get list of documents that contain the words within a given proximity window
def getDocument(word1, word2, k):
    commons = set(unigram_index[word1].keys()) & set(unigram_index[word2].keys())  # get documents common to both words
    inv_list = {}
    res = []
    inv_list[word1] = unigram_index[word1]        # get inverted list for word 1
    inv_list[word2] = unigram_index[word2]        # get inverted list for word 2
    for doc in commons:
        pos1 = getposition(inv_list[word1][doc])    # get positions for word 1
        pos2 = getposition(inv_list[word2][doc])    # get positions for word 2
        for idx in pos1:
            prox = range(idx-k, idx+k+1)              # for each position of word1 in doc check if word2 is within k positions
            if len(set(prox) & set(pos2)):          
                res.append(doc)                      # if word2 is within k positions append docid to the result list
                break
    return res

In [406]:
query = "space mission"            # for query string 'space mission'
word_list = []
for token in ngrams(word_tokenize(query), 1):       # tokenize the query - for matching the index terms
    word_list.append(token)
doc1 = getDocument(word_list[0], word_list[1], 6)    # get document list within k = 6 positions
doc2 = getDocument(word_list[0], word_list[1], 12)   # get document list within k = 12 positions

In [407]:
query = "earth orbit"                        # for query string 'earth orbit'
word_list = []
for token in ngrams(word_tokenize(query), 1):          # tokenize the query
    word_list.append(token)
doc3 = getDocument(word_list[0], word_list[1], 5)        # get document list within k = 5 positions
doc4 = getDocument(word_list[0], word_list[1], 10)       # get document list within k = 10 positions

### write the document lists to files
with open("space_mission_12.txt", 'w+') as myfile:
    myfile.write(str(doc2))
myfile.close()


## Task 3 (25 points): Compute Corpus Statistics; Propose Stop Lists
a) For each inverted index in Task 1-a), generate a term frequency table comprising of two
columns: term and term frequency (for the corpus).
Sort the table from most to least frequent.
b) For each inverted index in Task 1-a), generate a document frequency table comprising of
three columns: term, docIDs, and document frequency. Sort lexicographically based on
term.
Note: For tasks 3-a) and 3-b), you will generate six tables in total: two tables for word
unigrams, two tables for word bigrams, and two tables for word trigrams.
c) Generate three stop lists, one per index from Task 1-a). How would you choose your
cutoff values? Briefly justify your choice and comment on the stop lists’ contents. 

In [360]:
"""
This function is used to compute the term frequency for the corpus for a given inverted index
"""
def termFrequency(index_ds):
    res = {}
    for term in index_ds.keys():      # for each term in the inverted index
        tot_freq = 0
        posting = index_ds[term]      # get the list of document ids and term frequency
        for i in range(len(posting)):
            if i%2 != 0:                 # since frequencies are stored in odd index positions
                tot_freq += posting[i]     # add the frequency to total frequency
        res[term] = tot_freq               # append to result
    return res

unigram_freq = termFrequency(unigram_tf)      # get term frequency for unigrams
bigram_freq = termFrequency(bigram_index)    # get term frequency for bigrams
trigram_freq = termFrequency(trigram_index)    # get term frequency for trigrams
        
    

In [361]:
# sort the term frequencies in descending order
sorted_unigrams = [(key, unigram_freq[key]) for key in sorted(unigram_freq, key=unigram_freq.__getitem__, reverse=True)]
sorted_bigrams = [(key, bigram_freq[key]) for key in sorted(bigram_freq, key=bigram_freq.__getitem__, reverse=True)]
sorted_trigrams = [(key, trigram_freq[key]) for key in sorted(trigram_freq, key=trigram_freq.__getitem__, reverse=True)]

In [413]:
# Used to dump the data structures to text files
f = open("trigram_term_frequency.txt", "w+") 
f.write(str(sorted_trigrams))
f.close()
f = open("unigram_term_frequency.txt", "w+")
f.write(str(sorted_unigrams))
f.close()
f = open("bigram_term_frequency.txt", "w+")
f.write(str(sorted_bigrams))
f.close()

In [376]:
"""
This function is used to retrieve the document list for each term 
"""
def documentFrequency(index_ds):
    res = {}
    for term in index_ds.keys():      # for each term in the index
        doc_ids = []
        posting = index_ds[term]          # get the inverted list
        for i, item in enumerate(posting):      # for each item in the posting
            if i%2 == 0:
                doc_ids.append(item)           # retrieve document id from even index positions
        res[str(term).strip("\'(),\'")] = doc_ids   # save the term and document list, strip the brackets from the term to aid sorting
    return res



bigram_doc = documentFrequency(bigram_index)            # generate table for bigrams
trigram_doc = documentFrequency(trigram_index)          # generate table for trigrams
unigram_doc = documentFrequency(unigram_tf)           # generate table for unigrams

In [378]:
# sort the terms lexicographically
unigram_doc = [(key, unigram_doc[key]) for key in sorted(unigram_doc.keys())]

bigram_doc = [(key, bigram_doc[key]) for key in sorted(bigram_doc.keys())]

trigram_doc = [(key, trigram_doc[key]) for key in sorted(trigram_doc.keys())]

### write the tables to files
workbook = xlsxwriter.Workbook('Trigram_Document_Frequency.xlsx')
worksheet = workbook.add_worksheet()
row = 1
col = 0
for term in trigram_doc:
    worksheet.write(row, col, term[0])
    worksheet.write(row, col+1, str(term[1]))
    worksheet.write(row, col+2, len(term[1]))
    row += 1
workbook.close()

In [466]:
# returns idf for a given posting since docs and term freq are in a single list, len(posting)/2 returns # of documents
# containing the word
def docfrequency(posting):
    return np.log(len(doc_tf_unigram)/(len(posting)/2)) 

uni_df = {}
for term in unigram_tf.keys():
    posting = unigram_tf[term]      # get the postings for each term
    idf = docfrequency(posting)     # compute idf 
    uni_df[term] = idf
        
    
uni_dataframe = pd.DataFrame.from_dict(uni_df, orient='index')         # convert the dictionary to data frame

uni_dataframe.sort_values(by = 0).head(100)                # sort the idf scores and select top 100 values

,0
"(of,)",0.003017
"(the,)",0.004024
"(a,)",0.009077
"(and,)",0.009077
"(in,)",0.010091
"(to,)",0.013138
"(as,)",0.027483
"(is,)",0.027483
"(with,)",0.033694
"(by,)",0.036814


In [450]:
uni_stop_list = uni_dataframe.sort_values(by=0).head(100).index  # save the top 100 terms in stop list


In [467]:
# generate the stop list for bigrams
bi_df = {}
for term in bigram_index.keys():
    term_tf = {}
    posting = bigram_index[term]
    idf = docfrequency(term, posting)
    bi_df[term] = idf
        
    
bi_dataframe = pd.DataFrame.from_dict(bi_df, orient='index')

bi_dataframe.sort_values(by = 0).head(100)

,0
"(of, the)",0.018238
"(in, the)",0.044132
"(to, the)",0.057867
"(and, the)",0.103577
"(from, the)",0.135254
"(on, the)",0.137556
"(by, the)",0.142174
"(with, the)",0.157335
"(of, a)",0.178714
"(to, be)",0.183527


In [455]:
bi_stop_list = bi_dataframe.sort_values(by=0).head(100).index



In [459]:
# generate the stop list for trigrams
tri_df = {}
for term in trigram_index.keys():
    term_tf = {}
    posting = trigram_index[term]
    idf = docfrequency(term, posting)
    tri_df[term] = idf
        
    
tri_dataframe = pd.DataFrame.from_dict(tri_df, orient='index')

tri_dataframe.sort_values(by = 0).head(100)

,0
"(one, of, the)",0.560870
"(as, well, as)",0.616892
"(part, of, the)",0.696161
"(due, to, the)",0.817976
"(such, as, the)",0.834013
"(known, as, the)",0.938604
"(a, number, of)",1.029820
"(the, end, of)",1.032633
"(was, the, first)",1.067020
"(as, a, result)",1.090620


In [460]:
tri_stop_list = tri_dataframe.sort_values(by = 0).head(100).index

### Write the stop lists to file
with open('bigram_stop_list.txt', 'w+') as myfile:
    myfile.write(str(bi_stop_list))
myfile.close()